# # Classifiers comparison on texts with naive Bayes assumption

In this session of laboratory we compare two models for categorical data probabilistic modeling: 
1. multivariate Bernoulli 
2. multinomial on a dataset 

We adopt a dataset on Twitter messages labelled with emotions (Joy vs Sadness).

The following program shows the loading of the data from a file.

Data are loaded into a matrix X adopting a sparse matrix representation, in order to save space and time.
Sparse matrix representation (in the csr format) represents in three "parallel" arrays the value of the matrix cells that are different from zero and the indices of those matrix cells.
The arrays are called: 
- data
- row
- col

- data[i] stores the value of the matrix cell #i whose indexes are contained in row[i] and col[i] 
- row[i] stores the index of the row in the matrix of the cell #i, 
- col[i] stores the index of the column of the cell #i.


The data file is in csv format.
Any Twitter message has been preprocessed by a Natural Language pipeline which eliminated stop words and substituted the interesting document elements with an integer identifier.  
The interesting document elements might be words, emoji or emoticons. The elements could be repeated in the same document and are uniquely identified in the documents by the same integer number.

Each row of the dataset is a list of integer number pairs, followed by a string which is the label of the document (Joy or sadness).
The first number of the pair is an identifier of an element (word, emoji or emoticon) and the second number of the pair is the count (frequency) of that element in that document.

The dataset has:
tot_n_docs=n_rows=11981
n_features (document elements)=11288



The following program reads the data file and loads in a sparse way the matrix using the scipy.sparse library

In [2]:

from numpy import ndarray, zeros
import numpy as np
import scipy
from scipy.sparse import csr_matrix

class_labels = ["Joy","Sadness"]
n_features=11288 # number of columns in the matrix = number of features (distinct elements in the documents)
n_rows=11981 # number rows of the matrix
n_elements=71474 # number of the existing values in the matrix (not empty, to be loaded in the matrix in a sparse way)

#path_training="/Users/meo/Documents/Didattica/Laboratorio-15-16-Jupyter/"
path_training="Datasets for Naive Bayes Classification/"
file_name="joy_sadness6000.txt"

# declare the row and col arrays with the indexes of the matrix cells (non empty) to be loaded from file
# they are needed because the matrix is sparse and we load in the matrix only the elements which are present
row=np.empty(n_elements, dtype=int)
col=np.empty(n_elements, dtype=int)
data=np.empty(n_elements, dtype=int)

row_n=0 # number of current row to be read and managed
cur_el=0 # position in the arrays row, col and data
twitter_labels=[] # list of class labels (target array) of the documents (twitter) that will be read from the input file
twitter_target=[] # list of 0/1 for class labels
with open(path_training + file_name, "r") as fi:
    for line in fi:
        el_list=line.split(',')
        l=len(el_list)
        last_el=el_list[l-1] # I grab the last element in the list which is the class label
        class_name=last_el.strip() # eliminate the '\n'
        twitter_labels.append(class_name)
        # twitter_labels contains the labels (Joy/Sadness); twitter_target contains 0/1 for the respective labels
        if (class_name==class_labels[0]):
           twitter_target.append(0)
        else:
           twitter_target.append(1)
        i=0 # I start reading all the doc elements from the beginning of the list
        while(i<(l-1)):
            element_id=int(el_list[i]) # identifier of the element in the document
            element_id=element_id-1 # the index starts from 0 (the read id starts from 1)
            i=i+1
            value_cell=int(el_list[i]) # make access to the following value in the file which is the count of the element in the documento 
            i=i+1
            row[cur_el]=row_n # load the data in the three arrays: the first two are the row and col indexes; the last one is the matrix cell value
            col[cur_el]=element_id
            data[cur_el]=value_cell
            cur_el=cur_el+1
        row_n=row_n+1
fi.close
print("final n_row="+str(row))
# loads the matrix by means of the indexes and the values in the three arrays just filled
twitter_data=csr_matrix((data, (row, col)), shape=(n_rows, n_features)).toarray()
print("resulting matrix:")
print(twitter_data)
print(twitter_labels)
print(twitter_target)


final n_row=[0 0 0 ... 0 0 0]
resulting matrix:
[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
['Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy'

Write a program in the following cell that splits the data matrix in training and test set (by random selection) and predicts the class (Joy/Sadness) of the messages on the basis of the words. 
Consider the two possible models:
multivariate Bernoulli and multinomial Bernoulli.
Find the accuracy of the models and test is the observed differences are significant.

In [3]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
X_train, X_test, y_train, y_test = train_test_split(twitter_data, twitter_target, test_size=0.30, random_state=42)

In [5]:
clf1 = BernoulliNB()
y_pred_multivariate = clf1.fit(X_train, y_train).predict(X_test)
acc = accuracy_score(y_test, y_pred_multivariate)
print("1) Accuracy Multivariato: {}".format(round(acc, 4)))

clf2 = MultinomialNB()
y_pred_multinomial = clf2.fit(X_train, y_train).predict(X_test)
acc = accuracy_score(y_test, y_pred_multinomial)
print("2) Accuracy Multinomiale: {}".format(round(acc, 4)))

1) Accuracy Multivariato: 0.9505
2) Accuracy Multinomiale: 0.9471


In [6]:
from sklearn.model_selection import cross_val_score
from scipy import stats
import pandas as pd

scores_a = cross_val_score(clf1, twitter_data, twitter_target, cv=10) # score=accuracy
print("1) Mean Accuracy Multivariato: {}".format(round(np.mean(scores_a),4)))
print(pd.Series(scores_a).describe())
scores_b = cross_val_score(clf2, twitter_data, twitter_target, cv=10) # score=accuracy
print("2) Mean Accuracy Multinomiale: {}".format(round(np.mean(scores_b),4)))
print(pd.Series(scores_b).describe())

1) Mean Accuracy Multivariato: 0.9528
count    10.000000
mean      0.952758
std       0.008068
min       0.936561
25%       0.949917
50%       0.951586
75%       0.956204
max       0.965776
dtype: float64
2) Mean Accuracy Multinomiale: 0.9507
count    10.000000
mean      0.950672
std       0.007993
min       0.937396
25%       0.947621
50%       0.949499
75%       0.953075
max       0.964942
dtype: float64


In [7]:
different_prediction = []
for i in range(len(y_test)):
    if y_pred_multivariate[i] != y_pred_multinomial[i]:
        different_prediction.append(i)
print("I due modelli hanno classificato diversamente {}/{} istanze.\n".format(len(different_prediction), len(y_test)))

print("-" * 43)
print("Multivariato \t Multinomiale \t True class")
print("-" * 43)
for pred in different_prediction:
    if y_pred_multivariate[pred] == y_test[pred]:
        print("\x1b[0;34m{}\x1b[0m \t\t {} \t\t {}".format(y_pred_multivariate[pred], y_pred_multinomial[pred], y_test[pred]))
    else:
        print("{} \t\t \x1b[0;34m{}\x1b[0m \t\t {}".format(y_pred_multivariate[pred], y_pred_multinomial[pred], y_test[pred]))

I due modelli hanno classificato diversamente 16/3595 istanze.

-------------------------------------------
Multivariato 	 Multinomiale 	 True class
-------------------------------------------
0 		 1 		 0
0 		 1 		 0
0 		 1 		 0
1 		 0 		 1
0 		 1 		 0
0 		 1 		 0
0 		 1 		 0
0 		 1 		 0
0 		 1 		 0
0 		 1 		 0
0 		 1 		 0
0 		 1 		 0
0 		 1 		 0
0 		 1 		 0
0 		 1 		 1
1 		 0 		 0


#### Paired t-test

In [8]:
# pip install mlxtend
from mlxtend.evaluate import paired_ttest_kfold_cv

In [50]:
# degrees of freedom k-1 (k=10)
# critical value 2.262
# alpha = 0.05
t, p = paired_ttest_kfold_cv(
    estimator1=clf1,
    estimator2=clf2,
    X=np.array(twitter_data),
    y=np.array(twitter_target),
    cv=10,
    shuffle=True
)

print('t statistic: %.3f' % t)
print('p value: %.3f' % p)

t statistic: 4.105
p value: 0.003


Dato che `abs(t) > critival_calue` rifiutiamo l'ipotesi nulla (i due modelli sono significativamente differenti)

Dato che `p <= alpha` rifiutiamo l'ipotesi nulla (i due modelli sono significativamente differenti)

In [10]:
from scipy import stats

t, p = stats.ttest_rel(scores_a, scores_b)
print('t statistic: %.3f' % t)
print('p value: %.3f' % p)

t statistic: 3.212
p value: 0.011


##### Esempio di t-test con altri classificatori

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from mlxtend.data import iris_data
from sklearn.model_selection import train_test_split
from mlxtend.evaluate import paired_ttest_kfold_cv

X, y = iris_data()
clf_l = LogisticRegression(random_state=1, max_iter=1000)
#clf_d = DecisionTreeClassifier(random_state=1, max_depth=1)
clf_d = DecisionTreeClassifier(random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

score1 = clf_l.fit(X_train, y_train).score(X_test, y_test)
score2 = clf_d.fit(X_train, y_train).score(X_test, y_test)

print('Logistic regression accuracy: %.2f%%' % (score1*100))
print('Decision tree accuracy: %.2f%%' % (score2*100))

t, p = paired_ttest_kfold_cv(
    estimator1=clf_l,
    estimator2=clf_d,
    X=X, y=y,
)

print('t statistic: %.3f' % t)
print('p value: %.3f' % p)

Logistic regression accuracy: 97.37%
Decision tree accuracy: 94.74%
t statistic: 0.000
p value: 1.000
